In [ ]:
# install lib
!pip install diffusers transformers accelerate safetensors pillow torch gradio imageio[ffmpeg] --quiet

In [ ]:
# import lib
import torch, os, uuid, random, imageio
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, EulerAncestralDiscreteScheduler
from PIL import Image, ImageEnhance
import gradio as gr

In [ ]:
# Device Setup
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float16 if DEVICE == "cuda" else torch.float32
print(f"⚡ Running on {DEVICE.upper()}")

# Scheduler
scheduler = EulerAncestralDiscreteScheduler.from_pretrained(
    "runwayml/stable-diffusion-v1-5", subfolder="scheduler"
)

# Pipelines
txt2img = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    scheduler=scheduler,
    torch_dtype=DTYPE,
    use_auth_token=HUGGING_FACE_TOKEN
).to(DEVICE)
txt2img.enable_attention_slicing()

img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    scheduler=scheduler,
    torch_dtype=DTYPE,
    use_auth_token=HUGGING_FACE_TOKEN
).to(DEVICE)
img2img.enable_attention_slicing()

# 🔹 Function: Generate Image + Animation
def generate_media(user_prompt):
    bad_prompt = (
        "lowres, blurry, distorted, deformed, bad anatomy, noisy, ugly, pixelated, "
        "oversaturated, cropped, broken details"
    )

    # Random seed
    seed = random.randint(11111, 99999)
    generator = torch.Generator(device=DEVICE).manual_seed(seed)

    # Step 1: Text → Image
    with torch.autocast("cuda") if DEVICE == "cuda" else torch.no_grad():
        base_img = txt2img(
            prompt=user_prompt,
            negative_prompt=bad_prompt,
            guidance_scale=12,
            num_inference_steps=50,
            height=768, width=768,
            generator=generator
        ).images[0]

    # Step 2: Animation Frames (progressive refinement)
    frames = []
    current = base_img
    for i in range(12):  # 12 frames
        strength = 0.45 + (i * 0.03)  # gradually stronger edits
        with torch.autocast("cuda") if DEVICE == "cuda" else torch.no_grad():
            new_img = img2img(
                prompt=user_prompt,
                negative_prompt=bad_prompt,
                image=current,
                strength=min(strength, 0.85),
                guidance_scale=13,
                num_inference_steps=40
            ).images[0]
            # Slight enhancements
            new_img = ImageEnhance.Contrast(new_img).enhance(1.2)
            new_img = ImageEnhance.Brightness(new_img).enhance(1.1)
            frames.append(new_img)
            current = new_img

    # Step 3: Save outputs
    os.makedirs("results", exist_ok=True)
    image_path = f"results/final_{seed}.png"
    video_path = f"results/anim_{seed}.mp4"

    # Save final image
    frames[-1].save(image_path)

    # Save animation
    imageio.mimsave(video_path, frames, fps=6)  # 6 fps short animation

    return frames[-1], image_path, video_path

In [ ]:
# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## AI Image + Animation Generator")
    gr.Markdown("Enter your creative prompt and get both **Image & Animation** outputs!")

    with gr.Row():
        user_prompt = gr.Textbox(label=" Prompt", lines=2, placeholder="e.g., A cyberpunk dragon flying in neon city")
        run_btn = gr.Button("Generate Media")

    out_img = gr.Image(label="Final Image", type="pil")
    out_img_file = gr.File(label="Download Image")
    out_vid_file = gr.File(label="Download Animation")

    run_btn.click(fn=generate_media, inputs=user_prompt, outputs=[out_img, out_img_file, out_vid_file])

demo.launch(share=True, debug=True)